# TP 2-3 : Branch-and-bound applied to a knapsack problem

### Initialisation (à faire une seule fois)

In [1]:
import Pkg; 
Pkg.add("GraphRecipes"); Pkg.add("Plots"); 
using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


### Récupération des données

In [173]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Procédure d'application des tests de sondabilités TA, TO et TR pour le cas de la relaxation linéaire

In [174]:
function TestsSondabilite_relaxlin(infaisable, cout, solutions_courant,valeurs_binaires_courant, BestProfit, Bestsol)
    TA, TO, TR = false, false, false
    if(infaisable)#Test de faisabilite
        TA=true
        println("TA")
    elseif(cout <= BestProfit) #Test d'optimalite
        TO=true
        println("TO")
    elseif( prod(abs.([round.(v, digits=0) for v in (solutions_courant)].-(solutions_courant)) .<= fill(10^-5, size(solutions_courant))) 
        ) #Test de resolution
        TR=true
        println("TR")
        #if (value(benef) >= BestProfit)
        if (cout >= BestProfit)
            Bestsol = [v for v in valeurs_binaires_courant]
            #BestProfit=value(benef)
            BestProfit= cout
        end
    else
        println("non sondable")
    end
    TA, TO, TR, Bestsol, BestProfit
end

TestsSondabilite_relaxlin (generic function with 1 method)

### Procédure de séparation et stratégie d'exploration permettant de se placer au prochain noeud à traiter

In [175]:

function SeparerNoeud_relaxlin(price_courant,weight_courant,capacity_courant,price_constant_courant,solutions_courant,indices_correspond_courant,valeurs_binaires_courant, listvars, listvals,list_indices_separation)
    # le noeud est non-sondable. Appliquer le critère de séparation pour le séparer en sous-noeuds 
    # et choisir un noeud-fils le plus à gauche   
    
    #find a fractionnal variable
    i, var = 1, 0
    while((i <= length(solutions_courant)) && (var==0))
        #if(varsshouldbebinary[i] ∉ listvars)
        if(abs(round(solutions_courant[i], digits=0) - (solutions_courant[i]) ) >= 10^-5)
            var=indices_correspond_courant[i]
            break
        end
        i+=1
    end
    
    #=
    #find most fractionnal variable ?
    i, var, maxfrac = -1, 0, 0.0
    for i in 1:length(varsshouldbebinary)
        if(abs(round(value(varsshouldbebinary[i]), digits=0) - value(varsshouldbebinary[i]) ) >= maxfrac) 
            #if a variable is more fractinonal
            var=varsshouldbebinary[i]
            maxfrac=abs(round(value(varsshouldbebinary[i]), digits=0) - value(varsshouldbebinary[i]) )
            #println(i, " ", var, " ", maxfrac)
        end
    end
    =#
    push!(list_indices_separation,i)
    capacity_new = capacity_courant - weight_courant[i]
    price_constant_new = price_constant_courant + price_courant[i]
    valeurs_binaires_courant[var] = 1.0
    deleteat!(indices_correspond_courant,i)
    deleteat!(price_courant,i)
    deleteat!(weight_courant,i)

    push!(listvars,var) #stocker l'identite de la variable choisie pour la séparation
    push!(listvals,1.0) #stocker la branche choisie, identifiee par la valeur de la variable choisie
    listvars, listvals,capacity_new,price_constant_new
end


function ExplorerAutreNoeud_relaxlin(price_courant,weight_courant,capacity_courant,price_constant_courant,indices_correspond_courant,valeurs_binaires_courant,price,weight,listvars, listvals, listnodes,list_indices_separation)
    #this node is sondable, go back to parent node then right child if possible
    
    stop=false
    capacity_new = capacity_courant
    price_constant_new = price_constant_courant
    #check if we are not at the root node
    if (length(listvars)>= 1)
        #go back to parent node
        var=pop!(listvars)
        theval=pop!(listvals)
        tmp=pop!(listnodes)
        indice_separation = pop!(list_indices_separation)
        insert!(price_courant,indice_separation,price[var])
        insert!(weight_courant,indice_separation,weight[var])
        insert!(indices_correspond_courant,indice_separation,var)
        capacity_new = capacity_courant + theval*weight[var]
        price_constant_new = price_constant_courant - theval*price[var]
        #go to right child if possible, otherwise go back to parent
        while( (theval==0.0) && (length(listvars)>= 1))
            var=pop!(listvars)
            theval=pop!(listvals)
            tmp=pop!(listnodes)
            indice_separation = pop!(list_indices_separation)
            insert!(price_courant,indice_separation,price[var])
            insert!(weight_courant,indice_separation,weight[var])
            insert!(indices_correspond_courant,indice_separation,var)
            capacity_new = capacity_courant + theval*weight[var]
            price_constant_new = price_constant_courant - theval*price[var]
        end
        if theval==1.0
            push!(list_indices_separation,indice_separation)
            valeurs_binaires_courant[var] = 0.0
            deleteat!(indices_correspond_courant,indice_separation)
            deleteat!(price_courant,indice_separation)
            deleteat!(weight_courant,indice_separation)
            push!(listvars,var)
            push!(listvals,0.0)
        else
            println("\nFINISHED")
            stop=true
        end
    else
        #the root node was sondable
        println("\nFINISHED")
        stop=true
    end
    listvars, listvals, listnodes,capacity_new,price_constant_new, stop 
end

ExplorerAutreNoeud_relaxlin (generic function with 1 method)

###  Création de la relaxation linéaire (= modèle associé au noeud 0): <span style="color:red"> SECTION A SUPPRIMER !!!! </span>

<span style="color:red"> Cette section est à commenter/supprimer et remplacer par vos propres calculs de bornes supérieures et autres, par exemple basées sur les bornes 1 et 2 vues en cours, ou d'autres calculs de bornes de votre choix/conception validés au préalable par votre encadrant/e de TP </span>

In [176]:
# set optimizer
  function optimizer(price_courant,weight_courant,capacity_courant,price_constant_courant,indices_correspond_courant,valeurs_binaires_courant)
    stop = false
    R_courant = price_courant./weight_courant
    indicesRcourant = sortperm(R_courant,rev=true)
    i = 1
    j = capacity_courant
    solutions_courant = zeros(length(price_courant))
    cout = price_constant_courant
    if (j < 0 )
        return true,0,solutions_courant,valeurs_binaires_courant
    elseif(price_courant == [])
        return false,cout,valeurs_binaires_courant,valeurs_binaires_courant
    end
    while(!stop)
        indice = indicesRcourant[i]
        if (weight_courant[indice] <= j)
            solutions_courant[indice] = 1
            cout = cout + price_courant[indice]
            j = j- weight_courant[indice]
            i = i + 1
        else
            solutions_courant[indice] = j/weight_courant[indice]
            cout = cout + solutions_courant[indice]*price_courant[indice]
            j = 0
            break
        end
        if ((i > length(price_courant)) || (j == 0))
            stop = true
        end
    end
    [valeurs_binaires_courant[indices_correspond_courant[i]] = trunc(Int,solutions_courant[i]) for i in 1:length(indices_correspond_courant)]
    return false,cout,solutions_courant,valeurs_binaires_courant
end


optimizer (generic function with 1 method)

In [177]:
function AfficherNotreModele(price_courant,weight_courant,capacity_courant,price_constant_courant,indices_correspond_courant)
    print("Max ")
    if (price_courant != [])
        print(price_courant[1])
        print(" x[")
        print(indices_correspond_courant[1])
        print("]")
        for i in 2:length(indices_correspond_courant)
            print(" + ")
            print(price_courant[i])
            print(" x[")
            print(indices_correspond_courant[i])
            print("]")
        end
    end
    print(" + ")
    print(price_constant_courant)
    
    println("\nSubject to ")
    if (weight_courant != [])
        print(weight_courant[1])
        print(" x[")
        print(indices_correspond_courant[1])
        print("]")
        for i in 2:length(indices_correspond_courant)
            print(" + ")
            print(weight_courant[i])
            print(" x[")
            print(indices_correspond_courant[i])
            print("]")
        end
    else
        print("0")
    end
    print(" <= ")
    print(capacity_courant)
end
    

AfficherNotreModele (generic function with 1 method)

In [178]:
function AfficherSolutions(solutions_courant,indices_correspond_courant)
    if (solutions_courant !=[])
        for i in 1:length(indices_correspond_courant)
            print("     x[")
            print(indices_correspond_courant[i])
            print("] = ")
            print(solutions_courant[i])
        end
    end
end        

AfficherSolutions (generic function with 1 method)

### Boucle principale : résoudre la relaxation linéaire, appliquer les tests de sondabilité, identifier le prochain noeud, répéter.

In [179]:

function SolveKnapInstance(filename)

        price, weight, capacity = readKnaptxtInstance(filename)
    
        #create the structure to memorize the search tree for visualization at the end
        trParentnodes=Int64[] #will store orig node of arc in search tree
        trChildnodes=Int64[] #will store destination node of arc in search tree
        trNamenodes=[] #will store names of nodes in search tree
         
        #intermediate structure to navigate in the search tree
        listvars=[]
        listvals=[]
        listnodes=[]

        BestProfit=-1
        Bestsol=[]
        
        price_courant = [i for i in price]
        weight_courant = [i for i in weight]
        capacity_courant = capacity
        price_constant_courant = 0
        indices_correspond_courant = [i for i in 1:length(price)] 
        list_indices_separation = []
        valeurs_binaires_courant = zeros(length(price))
        
        current_node_number=0
        stop = false

        while(!stop)

            println("\nNode number ", current_node_number, ": \n-----\n")
            AfficherNotreModele(price_courant,weight_courant,capacity_courant,price_constant_courant,indices_correspond_courant)

            #Update the search tree
            push!(trNamenodes,current_node_number+1) 
            if(length(trNamenodes)>=2)
                push!(trParentnodes,listnodes[end]+1) # +1 because the 1st node is "node 0"
                push!(trChildnodes, current_node_number+1) # +1 because the 1st node is "node 0"
            end
            push!(listnodes, current_node_number)


            print("\nSolve Notre model to compute the bounds of the current node: start ... ")
            infaisable,cout,solutions_courant,valeurs_binaires_courant = optimizer(price_courant,weight_courant,capacity_courant,price_constant_courant,indices_correspond_courant,valeurs_binaires_courant)
            println("... end")

            print("\nSolution relax lin"); 
            if(infaisable)#(has_values(model2))
                print(" : NOT AVAILABLE (probably infeasible or ressources limit reached)")
            else
                AfficherSolutions(solutions_courant,indices_correspond_courant) 
            end
            println(" "); println("\nPrevious Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")

            TA, TO, TR, Bestsol, BestProfit = TestsSondabilite_relaxlin(infaisable, cout, solutions_courant,valeurs_binaires_courant, BestProfit, Bestsol)

            is_node_sondable = TA || TO || TR

            if(!is_node_sondable)
                listvars, listvals,capacity_courant,price_constant_courant = SeparerNoeud_relaxlin(price_courant,weight_courant,capacity_courant,price_constant_courant,solutions_courant,indices_correspond_courant,valeurs_binaires_courant, listvars, listvals,list_indices_separation)
            else
                listvars, listvals, listnodes,capacity_courant,price_constant_courant, stop = ExplorerAutreNoeud_relaxlin(price_courant,weight_courant,capacity_courant,price_constant_courant,indices_correspond_courant,valeurs_binaires_courant,price,weight,listvars, listvals, listnodes,list_indices_separation)
            end

            current_node_number = current_node_number + 1
        end

        println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)

        return BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes
    

end


SolveKnapInstance (generic function with 1 method)

### Affichage du résultat final

In [180]:
@time BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes = SolveKnapInstance("instancesETU/KNAPnewformat/test.opb.txt")
println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)
graphplot(trParentnodes, trChildnodes, names=trNamenodes, method=:tree)


Node number 0: 
-----

Max 42 x[1] + 40 x[2] + 12 x[3] + 25 x[4] + 0
Subject to 
7 x[1] + 4 x[2] + 3 x[3] + 5 x[4] <= 10
Solve Notre model to compute the bounds of the current node: start ... ... end

Solution relax lin     x[1] = 0.8571428571428571     x[2] = 1.0     x[3] = 0.0     x[4] = 0.0 

Previous Solution memorized Any[] with bestprofit -1

non sondable

Node number 1: 
-----

Max 40 x[2] + 12 x[3] + 25 x[4] + 42
Subject to 
4 x[2] + 3 x[3] + 5 x[4] <= 3
Solve Notre model to compute the bounds of the current node: start ... ... end

Solution relax lin     x[2] = 0.75     x[3] = 0.0     x[4] = 0.0 

Previous Solution memorized Any[] with bestprofit -1

non sondable

Node number 2: 
-----

Max 12 x[3] + 25 x[4] + 82
Subject to 
3 x[3] + 5 x[4] <= -1
Solve Notre model to compute the bounds of the current node: start ... ... end

Solution relax lin : NOT AVAILABLE (probably infeasible or ressources limit reached) 

Previous Solution memorized Any[] with bestprofit -1

TA

Node num

UndefVarError: [91mUndefVarError: graphplot not defined[39m

In [172]:
list_dossiers = readdir("instancesETU/KNAPnewformat")
deleteat!(list_dossiers,10)
println(list_dossiers)
for j in 1:length(list_dossiers)
    i = list_dossiers[j]
    list_fichiers = readdir("instancesETU/KNAPnewformat/"*i)
    println(i*" : ")
    println(list_fichiers)
    for k in 1:length(list_fichiers)
        f = list_fichiers[k]
        if ((findfirst("_20_",f) != nothing) || ((findfirst("_50_",f) != nothing) && ((findfirst("_5_",f) == nothing))))
            println(f)
            @time BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes = SolveKnapInstance("instancesETU/KNAPnewformat/"*i*"/"*f)
        end
    end
end

["almost_strongly_correlated", "circle", "inverse_strongly_correlated", "multiple_strongly_correlated", "profit_ceiling", "similar_weights", "strongly_correlated", "strongly_correlated_span", "subset_sum", "uncorrelated", "uncorrelated_span", "weakly_correlated", "weakly_correlated_span"]
almost_strongly_correlated : 
["knapPI_5_10000_10000_1_-1484157.opb.txt", "knapPI_5_10000_10000_2_-2405704.opb.txt", "knapPI_5_10000_10000_3_-3195902.opb.txt", "knapPI_5_10000_10000_4_-4012574.opb.txt", "knapPI_5_10000_10000_5_-4687981.opb.txt", "knapPI_5_10000_1000_1_-145393.opb.txt", "knapPI_5_10000_1000_2_-238597.opb.txt", "knapPI_5_10000_1000_3_-320932.opb.txt", "knapPI_5_10000_1000_4_-396116.opb.txt", "knapPI_5_10000_1000_5_-466744.opb.txt", "knapPI_5_1000_10000_1_-146390.opb.txt", "knapPI_5_1000_10000_2_-243637.opb.txt", "knapPI_5_1000_10000_3_-322069.opb.txt", "knapPI_5_1000_10000_4_-393112.opb.txt", "knapPI_5_1000_10000_5_-468478.opb.txt", "knapPI_5_1000_1000_1_-14202.opb.txt", "knapPI_5_1000_

FINISHED

******

Optimal value = 10417.0
  0.295812 seconds (561.88 k allocations: 95.068 MiB, 19.89% gc time)
knapPI_4_50_1000_1_-994.opb.txt

FINISHED

******

Optimal value = 994
  0.004961 seconds (445 allocations: 23.670 KiB)
knapPI_4_50_1000_2_-997.opb.txt

FINISHED

******

Optimal value = 997
  0.005249 seconds (715 allocations: 40.033 KiB)
knapPI_4_50_1000_3_-998.opb.txt

FINISHED

******

Optimal value = 998
  0.019188 seconds (1.08 k allocations: 62.629 KiB)
knapPI_4_50_1000_4_-999.opb.txt

FINISHED

******

Optimal value = 999
  0.023018 seconds (1.53 k allocations: 78.523 KiB)
multiple_strongly_correlated : 
["knapPI_14_10000_1000_1_-270697.opb.txt", "knapPI_14_10000_1000_2_-407357.opb.txt", "knapPI_14_10000_1000_3_-537473.opb.txt", "knapPI_14_10000_1000_4_-634984.opb.txt", "knapPI_14_10000_1000_5_-726720.opb.txt", "knapPI_14_1000_1000_1_-26616.opb.txt", "knapPI_14_1000_1000_2_-38335.opb.txt", "knapPI_14_1000_1000_3_-54883.opb.txt", "knapPI_14_1000_1000_4_-63934.opb.txt",


FINISHED

******

Optimal value = 997.0
  0.279449 seconds (509.21 k allocations: 88.404 MiB, 20.62% gc time)
strongly_correlated : 
["knapPI_3_10000_10000_1_-1492410.opb.txt", "knapPI_3_10000_10000_2_-2384739.opb.txt", "knapPI_3_10000_10000_3_-3189248.opb.txt", "knapPI_3_10000_10000_4_-3972691.opb.txt", "knapPI_3_10000_10000_5_-4701785.opb.txt", "knapPI_3_10000_1000_1_-146919.opb.txt", "knapPI_3_10000_1000_2_-239710.opb.txt", "knapPI_3_10000_1000_3_-322125.opb.txt", "knapPI_3_10000_1000_4_-397431.opb.txt", "knapPI_3_10000_1000_5_-467270.opb.txt", "knapPI_3_1000_10000_1_-147980.opb.txt", "knapPI_3_1000_10000_2_-240124.opb.txt", "knapPI_3_1000_10000_3_-319482.opb.txt", "knapPI_3_1000_10000_4_-393174.opb.txt", "knapPI_3_1000_10000_5_-473215.opb.txt", "knapPI_3_1000_1000_1_-14390.opb.txt", "knapPI_3_1000_1000_2_-24196.opb.txt", "knapPI_3_1000_1000_3_-32480.opb.txt", "knapPI_3_1000_1000_4_-39767.opb.txt", "knapPI_3_1000_1000_5_-46333.opb.txt", "knapPI_3_100_10000_1_-23982.opb.txt", "knapP


FINISHED

******

Optimal value = 9955.0
  0.073178 seconds (571.31 k allocations: 38.224 MiB)
knapPI_6_50_10000_3_-9743.opb.txt

FINISHED

******

Optimal value = 9743.0
  0.073127 seconds (293.08 k allocations: 29.064 MiB)
knapPI_6_50_10000_4_-10437.opb.txt

FINISHED

******

Optimal value = 10437.0
  0.057425 seconds (167.00 k allocations: 21.433 MiB)
knapPI_6_50_1000_1_-994.opb.txt

FINISHED

******

Optimal value = 994.0
  0.011107 seconds (2.66 k allocations: 211.529 KiB)
knapPI_6_50_1000_2_-997.opb.txt

FINISHED

******

Optimal value = 997.0
  0.009076 seconds (19.87 k allocations: 1.818 MiB)
knapPI_6_50_1000_3_-998.opb.txt

FINISHED

******

Optimal value = 998.0
  0.022993 seconds (18.06 k allocations: 2.196 MiB)
knapPI_6_50_1000_4_-999.opb.txt

FINISHED

******

Optimal value = 999.0
  0.010620 seconds (13.49 k allocations: 2.236 MiB)
uncorrelated : 
["knapPI_1_10000_10000_1_-5677675.opb.txt", "knapPI_1_10000_10000_2_-8334617.opb.txt", "knapPI_1_10000_10000_3_-9807732.opb.t

InterruptException: [91mInterruptException:[39m